In [1]:
import praw
import nltk

# Function for Implementing Unix to year

In [2]:
def unix_year(time):
    
  month = ((time/86400)//365)
  year = 1970 + int(month)
  return year

# # Fetch Data


In [3]:
def fetch_data(lim = 1000,subreddit = "AskReddit"):

    reddit = praw.Reddit(client_id = 'nz04TlX3jb7wHw', client_secret = 'nXjxefn8FSkCH04dxihWFlFdOcTFrA',
                     username = 'Trapnova',password = 'ayush@flash121',
                     user_agent = 'prawTutorial')

    subreddit = reddit.subreddit(subreddit)
    hot_python = subreddit.top(limit = lim)

    data = {"title" : [],
        "question" : [],
        "score" : [],
        "date" : [],
        "num_comments" : [],
        "score_ratio" : [] }


    for submission in hot_python:
    
        if not submission.stickied:
            
            data["question"].append(submission.title)
            data["title"].append(submission.title.split())
            data["score"].append(submission.score)
            data["date"].append(unix_year(submission.created))
            data["num_comments"].append(submission.num_comments)
            data["score_ratio"].append(submission.upvote_ratio)

    return data

# # Clean Text

In [4]:
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
def clean_text(dataset):

    stop_words = set(stopwords.words("english"))
    punctuation = '!#$%&\"()*+,-./:;<=>?@[\\]^_`{|}~'
    length = len(dataset["title"])

    for i in range(length):

        x = []
        for j in range(len(dataset["title"][i])):

            if dataset["title"][i][j] in stop_words:
                pass
            else:
                x.append(dataset["title"][i][j].translate(str.maketrans("","",punctuation)))

        dataset["title"][i] = x



In [6]:
def lemmatize_text(dataset):

    wordnet_lemmatizer = WordNetLemmatizer()

    for i in range(len(dataset["title"])):
        for j in range(len(dataset["title"][i])):
            dataset["title"][i][j] = dataset["title"][i][j].replace(",","")
            dataset["title"][i][j] = wordnet_lemmatizer.lemmatize(dataset["title"][i][j],pos = 'v')

# #  Pandas

In [7]:
import csv
import pandas as pd

In [8]:
def pandas(dataset):
    dataframe = pd.DataFrame(dataset)
    dataframe = dataframe.drop(columns = "question")
    return dataframe

# # MLB

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer

In [11]:
def mlb_text(dataset,dataframe):

    labels = []
    for i in dataset["title"]:
        for j in i:
            if j in labels:
                pass
            else:
                labels.append(j)

    mlb = MultiLabelBinarizer(classes = labels)

    hot_encoded = mlb.fit_transform(dataframe["title"])
    column = pd.DataFrame(hot_encoded,columns=labels)
    dataframe = pd.concat([dataframe,column],axis=1)

    return dataframe

# # CSV

In [12]:
def pd2csv(dataframe):

    dataframe.to_csv("data.csv",index=False)

In [13]:
dict1 = fetch_data(900,"AskReddit")
clean_text(dict1)
lemmatize_text(dict1)
df = pandas(dict1)
df = mlb_text(dict1,df)
pd2csv(df)

In [ ]:
dfa = pd.read_csv("/home/jovyan/work/data.csv")